# 字典

## 字典结构与dict get命令

字典是类似有偶数个元素的列表，其中奇数位置的元素作为key(互不相同)，其后的偶数位置元素作为value。

In [1]:
set example {firstname Joe surname Schmoe title Mr}
puts [dict get $example firstname]

Joe


In [2]:
set prefers {
    Joe {the easy life}
    Jeremy {fast cars}
    {Uncle Sam} {motherhood and apple pie}
}
puts [dict get $prefers Joe]
puts [dict get $prefers {Uncle Sam}]

the easy life
motherhood and apple pie


In [3]:
set employees {
    0001 {
        firstname Joe
        surname Schmoe
        title Mr
    }
    1234 {
        firstname Ann
        initial E
        surname Huan
        title Miss
    }
}
puts [dict get $employees 1234 firstname]

Ann


**小结**：`dict get 字典变量 key1 key2 ...`，按照嵌套的层级结构取出key对应的value

## 创建和更新字典

>字典中的key的顺序是固定的。


字典创建：`dict create`

字典更新：`dict set; dict replace`<br>
`set example [dict create firstname Ann initial E surname Huan]`<br>
* `dict replace $example initial Y`：更新已存在key的value。`$example`的值并没有改变，要想改变的话需要使用set命令重新为example赋值，即`set example [dict replace $example initial Y]`<br>
* `dict replace $example title Mrs surname Boddie`：添加一个键值对。添加的项数必须是偶数，组成键值对的形式。<br>
* `dict remove $example initial`：删除关键字initial，关键字可以不存在，也可以是多个。<br>

In [4]:
set example [dict create firstname Ann initial E surname Huan]
puts $example
puts [dict replace $example initial Y]
puts [dict replace $example title Mrs surname Boddie]
puts [dict remove $example initial firstname key]

firstname Ann initial E surname Huan
firstname Ann initial Y surname Huan
firstname Ann initial E surname Boddie title Mrs
surname Huan


合并字典：`dict merge`
>参数是两个或多个字典变量。若几个字典变量存在相同的key，合并后的字典中该key对应的value是按照参数顺序最后一个字典变量出现的value。

In [5]:
set color1 {foreground white background black}
set color2 {highlight red foreground green}
set color3 {foreground blue underline True}
puts [dict merge $color1 $color2 $color3]

foreground blue background black highlight red underline True


`dict set`：添加关键字，改变关键字的映射。**它与dict replace的区别是，它会写回原来的变量，而dict replace需要重新赋值。**<br>
`dict unset`：删除关键字。

In [6]:
set example [dict create firstname Ann initial E surname Huan title Miss]
puts $example
dict set example title Mrs
puts $example
dict set example surname Boddie
puts $example 
dict unset example initial
puts $example

firstname Ann initial E surname Huan title Miss
firstname Ann initial E surname Huan title Mrs
firstname Ann initial E surname Boddie title Mrs
firstname Ann surname Boddie title Mrs


## 检测字典

`dict size`：返回字典的key-value的个数<br>
`dict exists`：检测是否存在某个key<br>
`dict keys`：以列表的形式返回所有的key<br>
`dict values`：以列表的形式返回所有的value<br>
`dict for`：遍历字典。<br>

In [7]:
set example [dict create firstname Ann surname Huan title Miss]
puts [dict size $example]
puts [dict exists $example firstname],[dict exists $example sex]
puts [dict keys $example]
puts [dict values $example]
dict for {key value} $example {
    puts [format "%s: %s" $key $value]
}

3
1,0
firstname surname title
Ann Huan Miss
firstname: Ann
surname: Huan
title: Miss


## 更新字典中的value值

`dict append`：在原来的值后面添加一个或多个字符串<br>
`dict lappend`：以元素的形式添加

In [8]:
set example {firstname Ann surname Huan title Miss}
dict append example firstname ie
puts $example
dict append example firstname Joe John
puts $example

firstname Annie surname Huan title Miss
firstname AnnieJoeJohn surname Huan title Miss


In [9]:
set shopping {fruit apple veg carrot}
puts $shopping
dict lappend shopping fruit orange
puts $shopping
dict lappend shopping veg cabbage beans
puts $shopping

fruit apple veg carrot
fruit {apple orange} veg carrot
fruit {apple orange} veg {carrot cabbage beans}


****

`dict incr`：它会为每个key生成一个整数的value，incr一下value就+1。主要用于统计文本中单词的个数。

In [10]:
set example {firstname Ann surname Huan}
dict for {key value} $example {
    dict incr example $key
}
puts $examplee

expected integer but got "Ann"

In [11]:
proc count {text} {
    set frequencies {}
    foreach word [split $text] {
        if {$word eq " "} continue
        dict incr frequencies [string tolower $word]
    }
    return $frequencies
}
puts [count "this day is a happy happy day"]

this 1 day 2 is 1 a 1 happy 2


In [12]:
set s "this day is a  happy happy day"
foreach v [split $s] {
    puts $v
}

this
day
is
a

happy
happy
day


****

`dict update`：字典更新集大成者。它可以索引出一个或多个key-value值，通过这个key-value进而达到对字典的修改。另外，更新只发生在dict update命令块结束的时候。

In [13]:
set example {firstname Ann surname Huan}
dict update example firstname v1 surname v2 {
    puts "$v1, $v2"
}

Ann, Huan


In [14]:
set example {firstname Ann surname Huan initial E}
dict update example firstname v1 surname v2 initial v3 {
    # 删除关键字initial
    unset v3
    puts $example
}
puts $example

firstname Ann surname Huan initial E
firstname Ann surname Huan


## 使用嵌套字典

支持嵌套字典的命令：`dict get, dict exists, dict set, dict unset, dict with`

`dict get $字典变量 key1 key2 ...`<br>
`dict exists $字典变量 key1 key2 ...`: 检查key1,key2...是否存在，路径中的各级关键字都应当是字典，否则报错。<br>
`dict set`：沿着必要的路径创建字典<br>
`dict unset`：删除路径中最里层的关键字。<br>

In [15]:
set example {firstname Ann surname Huan}
puts $example 
dict set example address street {Ordinary Way}
puts $example
dict unset example address street
puts $example

set example {firstname Ann surname Huan}
dict set example address home school stinghua class No1
puts $example

firstname Ann surname Huan
firstname Ann surname Huan address {street {Ordinary Way}}
firstname Ann surname Huan address {}
firstname Ann surname Huan address {home {school {stinghua {class No1}}}}


****

`dict with`：允许把字典“打开”到变量。

In [16]:
set example {
    A {
        alphabet {a alpha b bravo c charlie}
        animals {cow calf sheep lamb pig ? goose ?}
    }
    C {
        comedians {laurel&hardy morecambe&wise}
    }
}
dict with example C {
    puts "comedians: $comedians"
    lappend comedians "steven martin"
}
puts [dict get $example C comedians]
dict with example A alphabet {
    puts "$a, $b, $c"
}
dict with example A animals {
    set pig piglet
    set goose gosling
}
dict with example A {
    dict for {key value} $animals {
        puts "$key has baby $value"
    }
}

comedians: laurel&hardy morecambe&wise
laurel&hardy morecambe&wise {steven martin}
alpha, bravo, charlie
cow has baby calf
sheep has baby lamb
pig has baby piglet
goose has baby gosling
